In [11]:
import pandas as pd
import asyncio
import getpass
from planet import Auth
import json
from planet_helpers import set_filters, parse_polygon
from datetime import datetime

DOWNLOAD_DIR = '../data/planet_data'

In [3]:
user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store()

In [4]:
events = pd.read_csv('../data/ACLED_Ukraine_events_sample.csv')[['location_id', 'event_date', 'event_id_cnty', 'timeline_id']]
non_events = pd.read_csv('../data/ACLED_Ukraine_non_events_sample.csv')[['location_id', 'event_date', 'timeline_id']]

places = pd.read_csv("../data/places.csv")

# merge events with places on location_id
events = events.merge(places, on='location_id')
non_events = non_events.merge(places, on='location_id')

In [5]:
# parse geometries
events['geometry'] = events['geometry'].apply(parse_polygon)

non_events['geometry'] = non_events['geometry'].apply(parse_polygon)

KeyboardInterrupt: 

In [ ]:
for event in events:
    geom =  {
       "type": "Polygon",
       "coordinates": event['geometry']
       }
    
    from_date = event['event_date']
    filters = set_filters(from_date, to_date, geom)

In [ ]:

places['geometry'] = places['geometry'].apply(parse_polygon)

geom = {
       "type": "Polygon",
       "coordinates": places['geometry'][0]
   }

date = datetime.strptime(events["event_date"][0], "%Y-%m-%d")
# subtract 5 days from the event date
five_before = date - pd.DateOffset(days=5).to_pydatetime()
five_after = date + pd.DateOffset(days=5).to_pydatetime()

filters = set_filters(five_before, five_after, geom)

In [37]:
for event in events.head().iterrows():
    print(event[2])

IndexError: tuple index out of range

In [32]:
all_filters = []
for event in events.head():
    
    coords = parse_polygon(event['geometry'][0])

    geom = {
       "type": "Polygon",
       "coordinates": coords
   }

    date = datetime.strptime(events["event_date"][0], "%Y-%m-%d")
    # subtract 5 days from the event date
    five_before = date - pd.DateOffset(days=5).to_pydatetime()
    five_after = date + pd.DateOffset(days=5).to_pydatetime()

    filters = set_filters(five_before, five_after, geom)

    all_filters.append(filters)

TypeError: string indices must be integers